## Initialization

In [2]:
#from Classes import Data, Assignment
    # Later, we can move the Class definitions into a separate file

In [3]:
%pip install pulp

# Install Gurobi
%pip install gurobipy 
    # Obtain academic license from: https://www.gurobi.com/downloads/end-user-license-agreement-academic/

# Install SCIP
%pip install pyscipopt

# Other useful packages
import numpy as np
import pandas as pd
import copy # To make deep copies
import os
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap
import seaborn as sns
from numpy.random import default_rng
from scipy.stats import norm
import time
from tqdm import tqdm # To show progress bar

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [4]:
# Check with solvers available on computer
import pulp as pl
from pulp import *
solver_list = pl.listSolvers(onlyAvailable=True)
print(solver_list)

['GUROBI', 'GUROBI_CMD', 'PULP_CBC_CMD', 'SCIP_CMD', 'FSCIP_CMD', 'SCIP_PY']


## Define classes
Define the following classes:
* 'Data': contains
    * Number of students
    * Number of schools
    * Preferences students
    * Preferences schools
    * Capacities schools
    * Names of students
    * Names of schools
    * File name
* 'Assignment': the selection probabilities of the students to the schools

In [6]:
class Data:
    # Define the initialization of an object from this class
    def __init__(self, n_stud: int, n_schools: int, pref: list, prior: list, cap:list, ID_stud:list, ID_school:list, file_name:str):
        self.n_stud = n_stud
        self.n_schools = n_schools
        self.pref = copy.deepcopy(pref)
        self.prior = copy.deepcopy(prior)
        self.cap = copy.deepcopy(cap)
        self.ID_stud = copy.deepcopy(ID_stud)
        self.ID_school = copy.deepcopy(ID_school)
        self.file_name = file_name   

        # Create alternative copies of pref and prior in which the elements are no longer strings, 
        # but the indices of the corresponding elements in the ID vectors
        self.pref_index = [[self.ID_school.index(school) for school in student_pref] for student_pref in self.pref]

        # Now create two matrices containing the position of the schools in the preferences, and of the students in the priorities
        # Initialize the rank matrix with NaN
        self.rank_pref = np.full((self.n_stud, self.n_schools), np.nan)

        self.prior_index = []
        for school_prior in self.prior:
            transformed_school_prior = []
            for student_group in school_prior:
                if isinstance(student_group, tuple):
                    transformed_school_prior.append(tuple(ID_stud.index(student) for student in student_group))
                else:
                    transformed_school_prior.append(ID_stud.index(student_group))
            self.prior_index.append(transformed_school_prior)

        
        # Populate the rank matrix
        for i, student_pref in enumerate(self.pref):
            for rank_position, school_id in enumerate(student_pref):
                if school_id in self.ID_school:
                    school_index = self.ID_school.index(school_id)
                    self.rank_pref[i][school_index] = rank_position

        # Initialize the rank_prior matrix with NaN
        self.rank_prior = np.full((self.n_schools, self.n_stud), np.nan)
        
        # Populate the rank_prior matrix
        for j, school_prior in enumerate(self.prior):
            for rank_position, student_id in enumerate(school_prior):
                # Handle tuple (grouped students) by expanding
                if isinstance(student_id, tuple):
                    for grouped_student in student_id:
                        if grouped_student in self.ID_stud:
                            student_index = self.ID_stud.index(grouped_student)
                            self.rank_prior[j][student_index] = rank_position + 1  # Positions are 1-based
                elif student_id in self.ID_stud:
                    student_index = self.ID_stud.index(student_id)
                    self.rank_prior[j][student_index] = rank_position + 1  # Positions are 1-based
    
    # Choose what is being shown for the command 'print(MyData)', where 'MyData' is an instance of the class 'Data'
    def __str__(self):
        s ="The data instance has the following properties: \n"
        s += f"\n\t{self.n_stud} students.\n\t{self.n_schools} schools. \n\n \tPREFERENCES:\n"
        for i in range(0,self.n_stud):
            s+= f"\t{self.ID_stud[i]}\t"
            for j in range(0, len(self.pref[i])):
                s+=f"{self.pref[i][j]} "
            s +="\n"

        s += f"\n\n \tCAPACITIES & PRIORITIES:\n"
        for i in range(0,self.n_schools):
            s+= f"\t{self.ID_school[i]}\t"
            s+= f"{self.cap[i]}\t"
            for j in range(0, len(self.prior[i])):
                if len(self.prior[i][j]) >= 2:
                    s+=f"{{"
                    for k in range(0, len(self.prior[i][j])):
                        s+=f"{self.prior[i][j][k]}"
                        if k < len(self.prior[i][j]) - 1:
                            s+= f" "
                    s+=f"}} "
                else:
                    s+=f"{self.prior[i][j]} "
            s +="\n"
        return s

In [7]:
class Assignment:
    # This class will contain an assignment
    def __init__(self, MyData: Data, p: np.ndarray, label = None):
        # self.file_name = MyData.file_name[:-4] 
            # Use this when importing .csv files, for example
        self.file_name = MyData.file_name
        self.MyData = copy.deepcopy(MyData)
        self.assignment = copy.deepcopy(p)
        self.label = label
        if label == None:
            self.label = ""
        
        names = []
        for i in range(0,MyData.n_stud):
            names.append("Choice {}".format(i + 1))
        
        # Same as assignment, but ranked in decreasing order of preference
        self.assignment_ranked = np.zeros(shape=(MyData.n_stud, MyData.n_schools), dtype = np.float64)
        counter =  0
        for i in range(0, MyData.n_stud):
            for j in range(0, len(MyData.pref[i])):
                
                # Convert pref[i][k] (school ID as string) to column index
                #col_index = int(MyData.pref[i][j]) - 1
                col_index = MyData.ID_school.index(MyData.pref[i][j])
                self.assignment_ranked[i][j] = self.assignment[i][col_index]
                counter += 1
        #self.assignment_ranked = pd.DataFrame(ranked, columns = names)

    
        # Export assignment
        self.export_assignment()
    
    # Visualize the assignment in different ways
    def visualize(self):
        # To export the figures, check if the correct folder exists:
        if os.path.exists("Results") == False:
            # If not, create folder
            os.makedirs("Results")
        
        s = os.path.join("Results", "Visualisations")
        if os.path.exists(s) == False:
            # If not, create folder
            os.makedirs(s)
        
        s = os.path.join("Results", "Visualisations",self.file_name)
        if os.path.exists(s) == False:
            os.makedirs(s)
            
        
        path = "Results/Visualisations/"
        # The assignment itself
        sns.set(rc = {'figure.figsize':(MyData.n_stud,MyData.n_schools/1.5)})
        
        # Create a custom colormap (to show negative values red)
        colors = ["red", "white", "blue"]  # Red for negatives, white for 0, blue for positives
        custom_cmap = LinearSegmentedColormap.from_list("CustomMap", colors)
        
        # Create the heatmap
        p = sns.heatmap(self.assignment, cmap = custom_cmap, center=0, annot=True, yticklabels = MyData.ID_stud, xticklabels = MyData.ID_school)
        p.set_xlabel("Students", fontsize = 15)
        p.set_ylabel("Schools", fontsize = 15)
        name = path + self.file_name + "/" + self.label + ".pdf"
        p.set_title(self.label, fontsize = 20)
        plt.savefig(name, format="pdf", bbox_inches="tight")
        
        # Assignment, ranked by preference
        plt.figure()

        # Create a custom colormap (to show negative values red)
        colors = ["red", "white", "green"]  # Red for negatives, white for 0, blue for positives
        custom_cmap2 = LinearSegmentedColormap.from_list("CustomMap", colors)
        
        # Create the heatmap
        sns.set(rc = {'figure.figsize':(MyData.n_stud,MyData.n_schools/1.5)})
        p = sns.heatmap(self.assignment_ranked, cmap = custom_cmap2, center=0, annot=True, yticklabels = MyData.ID_stud, xticklabels = range(1,MyData.n_schools + 1))
        p.set_xlabel("Preference", fontsize = 15)
        p.set_ylabel("Students", fontsize = 15)
        name = path + self.file_name + "/" + self.label + "_Ranked.pdf"
        title = self.file_name + ": ranked by decreasing preference"
        p.set_title(title, fontsize = 20)
        plt.savefig(name, format="pdf", bbox_inches="tight")
        
        plt.figure()
    
    # Save the assignment to the correct subdirectory
    def export_assignment(self):
        if os.path.exists("Results") == False:
            # If not, create folder
            os.makedirs("Results")

        s = os.path.join("Results", "Assignments")
        if os.path.exists(s) == False:
            # If not, create folder
            os.makedirs(s)

        s = os.path.join("Results", "Assignments",self.file_name)
        if os.path.exists(s) == False:
            os.makedirs(s)
        
        name = "Results/Assignments/" + self.file_name + "/" + self.label + "_" + self.file_name + ".csv"
        np.savetxt(name, self.assignment, delimiter=",")
        
    # Choose what is being shown for the command 'print(Sol)', where 'Sol' is an instance of the class 'Assignment'
    def __str__(self):
        
        return s
        

In [8]:
class Model: 
    """
    Contains two methods:
        __init__: initializes the model, and the solver environment

        Solve: solves the model.
            The parameters of this method can control which objective function is optimized, and which solver is used
    """
    
    # Used this example as a template for Pulp: https://coin-or.github.io/pulp/CaseStudies/a_sudoku_problem.html
    
    def __init__(self, MyData: Data, p: Assignment, print_out: bool, nr_matchings = -1):
        """
        Initialize an instance of Model.

        Args:
            MyData (type: Data): instance of class Data.
            p (type: Assignment): instance of class Assignment.
            print_out (type: bool): boolean that controls which output is printed.
            nr_matchings (optional): number of matchings used in the decomposition, optional parameter that defaults to n_students * n_schools + 1

        """
        # 'nr_matchings' refers to number of matchings used to find decomposition
        self.MyData = copy.deepcopy(MyData)
        self.p = copy.deepcopy(p)
        self.nr_matchings = nr_matchings
        if nr_matchings == -1:
            self.nr_matchings = self.MyData.n_stud * self.MyData.n_schools + 1

        # Create the pulp model
        self.model = LpProblem("Improving_ex_post_stable_matchings", LpMinimize)

        # Create variables to store the solution in
        self.Xdecomp = [] # Matchings in the found decomposition
        self.Xdecomp_coeff = [] # Weights of these matchings
        zero = np.zeros(shape=(self.MyData.n_stud, self.MyData.n_schools))
        self.Xassignment = Assignment(MyData, zero) # Contains the final assignment found by the model

        #### DECISION VARIABLES ####
        self.STUD = range(0,self.MyData.n_stud)
        self.SCHOOLS = range(0, self.MyData.n_schools)
        self.N_MATCH = range(0, self.nr_matchings)

        # Tuple with all student-school pairs that are preferred to outside option
        # This tuple contains the INDICES of the students and the pairs, and not their original names
        self.PAIRS = []
        for i in range(0, MyData.n_stud):
            for j in range(0,len(MyData.pref[i])):
                # Convert pref[i][k] (school ID as string) to column index
                col_index = MyData.ID_school.index(MyData.pref[i][j])
                self.PAIRS.append((i,col_index))   
        
        # M[k][i][j] = 1 if student i is assigned to school j in matching k, and 0 otherwise
        self.M = LpVariable.dicts("M", [(k, i, j) for k in self.N_MATCH for i, j in self.PAIRS], cat="Binary")

        # Auxiliary variables to avoid non-linearity
        self.z = LpVariable.dicts("z", [(k, i, j) for k in self.N_MATCH for (i, j) in self.PAIRS], 0, 1)

        # Rename M and z
        for k, i, j in self.M:
            student_name = self.MyData.ID_stud[i]
            school_name = self.MyData.ID_school[j]
            self.M[k, i, j].name = f"M_{k}_{student_name}_{school_name}"
            self.z[k, i, j].name = f"z_{k}_{student_name}_{school_name}"

        # Q[i][j] is the new probability with which student i is assigned to school j, lies between 0 and 1
        self.Q = LpVariable.dicts("q", self.PAIRS, 0, 1) 
    
        # w[k] is the weight of matching k in the decomposition
        self.w = LpVariable.dicts("w", self.N_MATCH, 0, 1)

        #### OBJECTIVE FUNCTION ####
            # Done separately in other functions (see function Solve)
        
            
        #### CONSTRAINTS ####
        # Other constraints defined for specific models in functions below (see function Solve)

        if print_out:
            print("Stability constraints...")
        # Stability
        for k in self.N_MATCH:
            for i in self.STUD:
                for j in range(len(self.MyData.pref_index[i])):
                    current_school = self.MyData.pref_index[i][j]
                    lin = LpAffineExpression()

                    lin += self.MyData.cap[current_school] * self.M[k, i, current_school]

                    # Add all schools that are at least as preferred as the j-ranked school by student i
                    for l in range(j):
                        lin += self.MyData.cap[current_school] * self.M[k,i,self.MyData.pref_index[i][l]]


                    # Add terms based on priorities
                    prior_current = self.MyData.rank_prior[current_school][i]
                    for s in self.STUD:
                        if s != i:
                            # If current_school ranks student s higher than student i
                            if self.MyData.rank_prior[current_school][s] <= self.MyData.rank_prior[current_school][i]:
                                if (s, current_school) in self.PAIRS:
                                    lin += self.M[k,s,current_school]

                    # Add to model:
                    name = "STAB_" + str(k) + "_" + str(self.MyData.ID_stud[i]) + "_" + str(self.MyData.ID_school[current_school]) 
                    self.model += (lin >= self.MyData.cap[current_school], name) 

        
        # Each student at most assigned to one school
        if print_out:
            print("Capacity constraints...")
        for l in self.N_MATCH:
            for i in self.STUD:
                self.model += lpSum([self.M[l,i,j] for j in self.SCHOOLS if (i,j) in self.PAIRS]) <= 1, f"LESS_ONE_{l,i}"

        # Capacities schools respected
        for l in self.N_MATCH:
            for j in self.SCHOOLS:
                self.model += lpSum([self.M[l,i,j] for i in self.STUD if (i,j) in self.PAIRS]) <= self.MyData.cap[j], f"LESS_CAP_{l,j}"
                                    

    def Solve(self, obj: str, solver: str, print_out: bool):
        """
        Solves the formulation.
        Returns an instance from the Assignment class.

        Args:
            obj (str): controls the objective function
                "IMPR_RANK": minimizes expected rank while maintaining ex-post stability
                "STABLE": maximizes fraction of stable matchings in decomposition
            solver (str): controls which solver is used. See options through following commands:
                solver_list = pl.listSolvers(onlyAvailable=True)
                print(solver_list)
            print_out (bool): boolean that controls which output is printed.

        """

        # Check that strings-arguments are valid

        # Valid values for 'solver'
        solver_list = pl.listSolvers(onlyAvailable=True)
        if solver not in solver_list:
           raise ValueError(f"Invalid value: '{solver}'. Allowed values are: {solver_list}")

        # Valid values for 'obj'
        obj_list = ["IMPR_RANK", "STABLE"]
        if obj not in obj_list:
           raise ValueError(f"Invalid value: '{obj}'. Allowed values are: {obj_list}")

        #### FORMULATION ####
        
        # Set the objective function
        if obj == "IMPR_RANK":
            self.Improve_rank(print_out)
        
        elif obj == "STABLE":
            self.Max_Stable_Fraction(print_out)

        self.model.writeLP("Test.lp")

        
        #### SOLVE ####
            
        # String can't be used as the argument in solve method, so convert it like this:
        solver_function = globals()[solver]  # Retrieves the GUROBI function or class
        
        # Solve the formulation
        self.model.solve(solver_function())
        #self.model.solve(GUROBI_CMD(keepFiles=True, msg=True, options=[("IISFind", 1)]))
        
        #### STORE SOLUTION ####
        # Make sure assignment is empty in Xassignment
        self.Xassignment.assignment = np.zeros(shape=(self.MyData.n_stud, self.MyData.n_schools))

        for (i,j) in self.PAIRS:
            self.Xassignment.assignment[i,j] = self.Q[i,j].varValue

        # Store decomposition
        self.Xdecomp = [] # Matchings in the found decomposition
        self.Xdecomp_coeff = [] # Weights of these matchings

        for l in self.N_MATCH:
            self.Xdecomp.append(np.zeros(shape=(self.MyData.n_stud, self.MyData.n_schools)))
            self.Xdecomp_coeff.append(self.w[l].varValue)
            for (i,j) in self.PAIRS:
                self.Xdecomp[-1][i,j] = self.M[l,i,j].varValue
                
        return self.Xassignment


    def Improve_rank(self, print_out: str):
        """
        Creates and solves formulation to minimize the expected rank while ensuring the found random matching is ex-post stable.
        """
        
        if print_out == True:
            # Compute average rank of current assignment

            sum = 0
            for (i,j) in self.PAIRS:
                sum += self.p.assignment[i,j] * (self.MyData.rank_pref[i,j] + 1) # + 1 because the indexing starts from zero
            # Average
            sum = sum/self.MyData.n_stud
            print(f"\nAverage rank before optimization: {sum}.\n\n")
        
        # Objective function
        lin = LpAffineExpression()
        for (i,j) in self.PAIRS:
            lin += (self.Q[i,j] * (self.MyData.rank_pref[i,j] + 1)) / self.MyData.n_stud # + 1 because the indexing starts from zero
        self.model += lin

        # Define q based on matchings in decomposition
            # Where z is an auxiliary variable to avoid non-linearities
        if print_out:
            print("Constraints with auxiliary z...")
        for l in self.N_MATCH:
            for (i,j) in self.PAIRS:
                self.model += self.z[l, i, j] - self.w[l] <= 0,f"z_w{l,i,j}" 

        for l in self.N_MATCH:
            for (i,j) in self.PAIRS:
                self.model += self.z[l, i, j] - self.M[l, i, j] <= 0,f"z_M_{l, i, j}"

        for l in self.N_MATCH:
            for (i,j) in self.PAIRS:
                self.model += self.z[l, i, j] + (1 - self.M[l, i, j]) - self.w[l]  >= 0,f"z_w_M_{l, i, j}"
                # Maybe these constraints are redundant because of the objective function

        for (i,j) in self.PAIRS:
            self.model += lpSum([self.z[l, i, j] for l in self.N_MATCH]) == self.Q[i,j], f"z_Q_{i, j}"

        # Ensure weights sum up to one
        self.model += lpSum([self.w[l] for l in self.N_MATCH]) == 1, f"SUM_TO_ONE"

        # First-order stochastic stability
        if print_out:
            print("FOSD constraints...")
        for i in self.STUD:
            for j in range(len(self.MyData.pref[i])):
                lin = LpAffineExpression()
                for k in range(j+1):
                    pref_school = self.MyData.pref_index[i][k]
                    lin += self.Q[i,pref_school]
                    lin -= self.p.assignment[i,pref_school]
                name = "FOSD_" +  str(self.MyData.ID_stud[i]) + "_" + str(j)
                self.model += (lin >= 0, name)


    def Max_Stable_Fraction(self, print_out: str):
        # Objective function
        obj = LpAffineExpression()
        for l in self.N_MATCH:
            obj += self.w[l] 
        self.model += obj
        self.model.sense = LpMaximize

        # Constraints to ensure that decomposition is at least equal to p (element-wise)
            # Where z is an auxiliary variable to avoid non-linearities
        if print_out:
            print("Constraints with auxiliary z...")
        for l in self.N_MATCH:
            for (i,j) in self.PAIRS:
                self.model += self.z[l, i, j] - self.w[l] <= 0,f"z_w{l,i,j}" 

        for l in self.N_MATCH:
            for (i,j) in self.PAIRS:
                self.model += self.z[l, i, j] - self.M[l, i, j] <= 0,f"z_M_{l, i, j}"

        for l in self.N_MATCH:
            for (i,j) in self.PAIRS:
                self.model += self.z[l, i, j] + (1 - self.M[l, i, j]) - self.w[l]  >= 0,f"z_w_M_{l, i, j}"
                # Maybe these constraints are redundant because of the objective function

        for (i,j) in self.PAIRS:
            self.model += lpSum([self.z[l, i, j] for l in self.N_MATCH]) == self.Q[i,j], f"z_Q_{i, j}"

        for (i,j) in self.PAIRS:
            self.model += lpSum([self.z[l, i, j] for l in self.N_MATCH]) <= self.p.assignment[i,j], f"z_p_{i, j}"
        

    def print_solution(self):
        s = "The obtained random matching is:\n"
        s+=f"\t\t"
        for j in self.SCHOOLS:
            s+=f"{self.MyData.ID_school[j]}\t"
        s+="\n"
        for i in self.STUD:
            s+= f"\t{self.MyData.ID_stud[i]}\t"
            for j in self.SCHOOLS:
                s+=f"{self.Xassignment.assignment[i,j]}\t"
            s+=f"\n"
        s+=f"\n"

        s+= "The matchings with positive weights are:\n"

        for l in self.N_MATCH:
            if self.Xdecomp_coeff[l] > 0:
                s+=f"\t w[{l}] = {self.Xdecomp_coeff[l]}\n"
                for i in self.STUD:
                    s+=f"\t\t"
                    for j in self.SCHOOLS:
                        if self.Xdecomp[l][i,j] == 1:
                            s+=f"1\t"
                        else:
                            s+= f"0\t"
                    s+=f"\n"
                s+=f"\n"
        print(s)

        
                
        

## Generate data

In [10]:
class DataGenParam:
    # Parameters to used for data generation, see https://github.com/DemeulemeesterT/GOSMI
    
    def __init__(self, capacity_ratio=1.2, corr_cap_pop=0.21, mean_pref=2.42, sigma_pref=1.05, 
                 CV_cap=0.8, CV_pop=0.6, delta_1=0.14, delta_2=0.009, pop_percentage=0.10):
        self.capacity_ratio = capacity_ratio
        self.corr_cap_pop = corr_cap_pop
        self.mean_pref = mean_pref
        self.sigma_pref = sigma_pref
        self.CV_cap = CV_cap
        self.CV_pop = CV_pop
        self.delta_1 = delta_1
        self.delta_2 = delta_2
        self.pop_percentage = pop_percentage

In [11]:
def generate_data(n_students: int, n_schools: int, parameters: DataGenParam, name: str, print_data=False, seed=123456789):
    """
    Generate data. The preferences and capacities are based on: https://github.com/DemeulemeesterT/GOSMI

    Parameters:
    - n_students: Number of students.
    - n_schools: Number of schools.
    - parameters: An instance of DataGenParam with generation parameters.
    - print_data: Whether to print the generated data.
    - seed: Random seed for reproducibility.

    Returns:
    - An object of the class Data
    """
    if seed != 123456789:
        # Use seed in argument
        rng = default_rng(seed)
    else:
        # Generate random seed 
        # Create a seed based on the current time
        seed = int(time.time() * 1000) % (2**32)  # Modulo 2^32 to ensure it's a valid seed

    # Initialize arrays
    students = list(range(n_students))
    schools = list(range(n_schools))

    # Generate capacities and popularity
    capacity_total = int(round(parameters.capacity_ratio * n_students))
    capacity_aid = rng.normal(0, 1, n_schools)
    popularity_aid = rng.normal(0, 1, n_schools)
    
    # Cholesky decomposition for correlated random variables
    covar = np.array([
        [1, parameters.corr_cap_pop],
        [parameters.corr_cap_pop, 1]
    ])
    G = np.linalg.cholesky(covar).T
    correlated = G @ np.vstack((capacity_aid, popularity_aid))
    capacity_aid, popularity_aid = correlated[0], correlated[1]

    # Rescale capacities
    mean_capacity_aid = np.mean(capacity_aid)
    capacities = np.round((capacity_total / n_schools) * 
                          (1 + parameters.CV_cap * (capacity_aid - mean_capacity_aid)))
    capacities = np.clip(capacities, 1, None).astype(int)  # Ensure no capacity < 1
    scale_factor = capacity_total / np.sum(capacities)
    capacities = np.round(capacities * scale_factor).astype(int)

    # Generate preference lengths
    pref_lengths = rng.normal(parameters.mean_pref, parameters.sigma_pref, n_students)
    pref_lengths = np.clip(pref_lengths, 1, n_schools).astype(int)

    # Determine popularity thresholds
    mean_pop_ratio = np.sum(pref_lengths) / np.sum(capacities)
    mean_popularity_aid = np.mean(popularity_aid)
    popularity_aid = mean_pop_ratio * (1 + parameters.CV_pop * (popularity_aid - mean_popularity_aid))
    popularity_aid = np.clip(popularity_aid, 0.2, None)  # Ensure no popularity < 0.2
    requests = capacities * popularity_aid
    popularity_aid *= np.sum(pref_lengths) / np.sum(requests)

    # Define popular schools
    sorted_indices = np.argsort(-popularity_aid)
    popularity_threshold = popularity_aid[sorted_indices[int(parameters.pop_percentage * n_schools)]]
    popular = popularity_aid > popularity_threshold

    # Generate preferences for each student
    preferences = []
    for i in range(n_students):
        student_pref = []
        available_schools = list(range(n_schools))
        for _ in range(pref_lengths[i]):
            weights = popularity_aid[available_schools]
            weights /= np.sum(weights)
            choice = rng.choice(available_schools, p=weights)
            student_pref.append(choice) 
            available_schools.remove(choice)
        preferences.append(student_pref)

    # Generate random school priorities:
    # For now, just simply
        # Randomly order students for each school
        # Divide into three indifference groups for each school

    priorities = []
    for j in range(n_schools):
        permutation = np.random.permutation(students)

        # Split the list into three roughly equal groups
        group_size = len(permutation) // 3
        group1 = permutation[:group_size]
        group2 = permutation[group_size:2 * group_size]
        group3 = permutation[2 * group_size:]

        priorities.append([tuple(group1), tuple(group2), tuple(group3)])
    
    # Optionally print data
    if print_data:
        print(f"Generated data with {n_students} students and {n_schools} schools.")
        print(f"Preferences: {preferences}")
        print(f"Priorities: {priorities}")
        print(f"Capacities: {list(capacities)}")
        # print(f"Popularity Ratios: {list(popularity_aid)}")
        print(f"Students: {students}")
        print(f"Schools: {schools}")
    
    MyData = Data(n_students, n_schools, preferences, priorities, capacities, students, schools, name)
    
    # Return results
    return MyData

## Gale-Shapley algorithm

In [13]:
def gale_shapley(MyData: Data):
    """
    Gale-Shapley algorithm.

    Parameters:
    - An instance from the Data class

    Returns:
    - A numpy array containing the assignment
    """
    n_stud = len(MyData.pref)
    n_schools = len(MyData.prior)
    pref = copy.deepcopy(MyData.pref) # We will gradually delete preferences from this 

    # Initialize data structures
    free_stud = list(range(n_stud))  # List of free students by index
    # Initialize temp_assigned with empty lists for each school
    temp_assigned = {school_index: [] for school_index in range(len(MyData.cap))} 

    while free_stud:
        # First we go through all students in 'free_stud' and remove them from the list
        while free_stud:
            i = free_stud[0]  # Get the first student
            free_stud.pop(0)  # Remove it
            # Assign free students to their most preferred school among remaining choices...
            # ... if preference list not empty yet
            if len(pref[i])>0:
                # Find index of that school
                index = MyData.ID_school.index(pref[i][0])
                temp_assigned[index].append(i) 
    
                # Remove that school from student i's preferences
                pref[i].pop(0)

        # Now each school j only keeps cap[j] most preferred students, and the others will be added to free_stud again
        for j in range(n_schools):
            if len(temp_assigned[j]) > MyData.cap[j]:
                # Dictionary containing priorities of the students who are temporarily assigned to school j
                prior_values = {stud_index: [] for stud_index in temp_assigned[j]}  
                for i in range(len(temp_assigned[j])):
                    # Find the position of student temp_assigned[j][i] in the priority list of school j
                    prior_values[temp_assigned[j][i]] = MyData.prior[j].index(MyData.ID_stud[temp_assigned[j][i]])

                # Sort the dictionary prior_values items by value
                sorted_prior_values = sorted(prior_values.items(), key=lambda item: item[1])

                # Remove the least preferred students who exceed capacity, and add them to free_students
                while len(temp_assigned[j]) > MyData.cap[j]:
                    # Add to free_stud
                    free_stud.append(sorted_prior_values[MyData.cap[j]][0])

                    # Remove from temp_assigned
                    temp_assigned[j].remove(sorted_prior_values[MyData.cap[j]][0])

                    # Remove from sorted_prior_values
                    sorted_prior_values.pop(MyData.cap[j])
    
    # Finally, transform the assignment in a numpy array where M[i][j] = 1 if student i is assigned to school j
    M = np.zeros(shape=(n_stud, n_schools))
    for j in range(n_schools):
        for k in range(len(temp_assigned[j])):
            M[temp_assigned[j][k]][j] = 1
    
    return M
    


## Sample Deferred Acceptance with tie-breaking

In [15]:
def DA_STB(MyData: Data, n_iter: int, seed = 123456789, print_out = False):
    """
    Deferred Acceptance with single tie-breaking

    Parameters:
    - MyData: An instance from the Data class
    - n_iter: number of tie-breakings sampled
    - print_out: boolean to control output on the screen

    Returns:
    - An instance of the Assignment class
    """

    if seed != 123456789:
        # Use seed in argument
        rng = default_rng(seed)
    else:
        # Generate random seed 
        # Create a seed based on the current time
        seed = int(time.time() * 1000) % (2**32)  # Modulo 2^32 to ensure it's a valid seed

    np.random.seed(seed)

    # First, check how many tie-breaking rules would be needed in total
    # Look at total number of students who are included in ties
    students_in_ties = set()
    for j in range(MyData.n_schools):
        for k in range(len(MyData.prior[j])):
            if len(MyData.prior[j][k]) >= 2: # When more than a single student in this element
                for l in range(len(MyData.prior[j][k])):
                    # students_in_ties.add(MyData.ID_stud.index(MyData.prior[j][k][l])) # We add the index of this student, not its name
                    students_in_ties.add(MyData.prior[j][k][l])

    students_in_ties = list(students_in_ties) # Convert the set to a list, allows us to access k-th element
    
    # The total number of needed tie-breaking rules is m!, where m = |student_in_ties|
    n_STB = math.factorial(len(students_in_ties))

    # We only need to perturb the students who appear in ties:
    
    if n_STB < n_iter:
        n_iter = n_STB
        # Enumerate all relevant permutations
        permut = list(itertools.permutations(students_in_ties))
    else:
        permut = set() # We first create a set, to ensure that all found permutations are unique. Later, convert to list
        # Sample n_iter out of all n_STB relevant permutations
        while len(permut) < n_iter:
            np.random.shuffle(students_in_ties)  # Shuffle in place
            permut.add(tuple(students_in_ties))
        permut = list(permut)
    
    if print_out:
        print(f"Students in ties: {len(students_in_ties)}")
        print(f"Tie-breaking rules needed: {n_STB}")
        print(f"Tie-breaking rules sampled: {n_iter}")
        # print(f"permut: {permut}")

    # For each of the permutations, break ties in the preferences and run Gale-Shapley algorithm on them
    M_sum = np.zeros(shape=(MyData.n_stud, MyData.n_schools)) # Will contain the final random_assignment

    for p in tqdm(permut):
        prior_new = [] 
        for j in range(len(MyData.prior)):
            # Just add priorities if no ties:
            if len(MyData.prior[j]) == MyData.n_stud:
                prior_new.append(MyData.prior[j])
            else:
                prior_array = []
                for k in range(len(MyData.prior[j])):
                    if len(MyData.prior[j][k]) == 1:
                        prior_array.append(MyData.prior[j][k])
                    else: # set of students who have same priorities
                        # Reorder the students based on the permuation
                        reordered_prior = list(sorted(MyData.prior[j][k], key=lambda x: p.index(x)))

                        # Add to prior_array
                        for l in range(len(MyData.prior[j][k])):
                            prior_array.append(reordered_prior[l])
                prior_new.append(prior_array)
                
        # Compute DA matching for the new priorities after tie-breaking
        Data_new_prior = Data(MyData.n_stud, MyData.n_schools, MyData.pref, prior_new, MyData.cap, MyData.ID_stud, MyData.ID_school, MyData.file_name)
        M_sum = M_sum + gale_shapley(Data_new_prior)            
        
    M_sum = M_sum / n_iter

    # Create an instance of the Assignment class
    label = MyData.file_name + "_" + "DA_STB" + str(n_iter)
    A = Assignment(MyData, M_sum, label)

    return A

    

## Initialize data

In [17]:
# Define preferences of the students
# 'pref[i][k]' contains the position of the k-th ranked school in the preferences.
# We assume the preferences to be strict
# Note that preferences can be strict. We indicate this by a tuple () in the list.

# Example paper
n_stud = 4
n_schools = 4

file_name = "Ex_paper"

# Preferences students
pref = [['1', '3', '4', '2'],
       ['1','4','3','2'],
       # ['1', '4'],
       ['2','3', '4', '1'],
       #['2', '4', '3', '1']]
        ['2', '4', '1', '3']]

# Priorities schools
prior = [[('A', 'B'), 'C', 'D'],
        [('C', 'D'), 'A', 'B'],
        ['B', 'D', ('A', 'C')],
        ['A', 'C', ('B', 'D')]]
#prior = [['D', 'A', 'B', 'C'],
#        ['C', 'D', 'A', 'B'],
#        ['B', 'D', 'A', 'C'],
#        ['A', 'C', 'B', 'D']]


# Capacities schools
cap = [1,1,1,1]

# Names of students and schools
ID_stud = ["A", "B", "C", "D"]
ID_school = ["1", "2", "3", "4"]

# Also create the random matching upon which we want to improve
p = np.zeros(shape=(n_stud, n_schools))
p[0][0] = 1/2
p[1][0] = 1/2
p[2][1] = 1/2
p[3][1] = 1/2
p[0][2] = 3/8
p[2][2] = 3/8
p[1][3] = 3/8
p[3][3] = 3/8
p[0][3] = 1/8
p[2][3] = 1/8
p[1][2] = 1/8
p[3][2] = 1/8

## Execution

In [19]:
parameters = DataGenParam(mean_pref = 8) # Default parameters
MyData = generate_data(n_students=20, n_schools=5, parameters = parameters, name="Test_DataGen", print_data=False, seed = 0)

In [20]:
# Uncomment next line for manually input data:
#MyData = Data(n_stud, n_schools, pref, prior, cap, ID_stud, ID_school, file_name)
print(MyData)

The data instance has the following properties: 

	20 students.
	5 schools. 

 	PREFERENCES:
	0	2 1 0 3 4 
	1	1 2 3 4 0 
	2	2 0 1 3 4 
	3	3 0 2 1 4 
	4	2 0 3 1 4 
	5	0 2 3 1 4 
	6	1 0 2 3 4 
	7	2 0 4 1 3 
	8	2 3 1 4 0 
	9	1 2 4 3 0 
	10	2 1 0 4 3 
	11	2 1 4 0 3 
	12	3 4 0 2 1 
	13	3 0 4 2 1 
	14	1 0 3 4 2 
	15	1 2 4 0 3 
	16	2 0 3 1 4 
	17	1 3 0 2 4 
	18	1 2 4 0 3 
	19	1 3 0 2 4 


 	CAPACITIES & PRIORITIES:
	0	5	{0 12 17 10 16 5} {3 2 8 4 11 7} {19 15 14 6 13 9 18 1} 
	1	5	{12 0 14 11 8 6} {9 4 10 5 18 17} {13 2 1 19 7 3 16 15} 
	2	8	{11 14 9 19 7 15} {13 0 10 2 1 4} {17 12 16 6 3 18 5 8} 
	3	4	{9 4 2 3 5 13} {10 0 12 18 11 15} {14 8 17 1 19 6 16 7} 
	4	1	{3 12 8 10 5 18} {7 15 16 2 14 9} {4 11 0 1 19 17 6 13} 



In [22]:
A = DA_STB(MyData, 1000, 0, True)
print(A.assignment)

Students in ties: 20
Tie-breaking rules needed: 2432902008176640000
Tie-breaking rules sampled: 1000


100%|██████████| 1000/1000 [00:01<00:00, 844.38it/s]

[[0.    0.025 0.975 0.    0.   ]
 [0.    0.    0.972 0.    0.028]
 [0.033 0.    0.967 0.    0.   ]
 [0.    0.    0.    1.    0.   ]
 [0.03  0.    0.97  0.    0.   ]
 [1.    0.    0.    0.    0.   ]
 [0.    1.    0.    0.    0.   ]
 [0.    0.    1.    0.    0.   ]
 [0.    0.508 0.    0.492 0.   ]
 [0.    0.851 0.149 0.    0.   ]
 [0.031 0.002 0.967 0.    0.   ]
 [0.    0.    1.    0.    0.   ]
 [0.    0.    0.    1.    0.   ]
 [0.    0.    0.    1.    0.   ]
 [0.    1.    0.    0.    0.   ]
 [0.    0.    1.    0.    0.   ]
 [1.    0.    0.    0.    0.   ]
 [0.17  0.809 0.    0.021 0.   ]
 [0.    0.805 0.    0.    0.195]
 [0.513 0.    0.    0.487 0.   ]]


In [683]:
A = Assignment(MyData, p, "Ex_paper")

# To visualize assignment
A.visualize()

IndexError: index 7 is out of bounds for axis 0 with size 4

In [ ]:
MyModel = Model(MyData, A, True)
q = MyModel.Solve("IMPR_RANK", "GUROBI", True)
#q = MyModel.Solve("STABLE", "GUROBI", True)

# The problem is solved using PuLP's choice of Solver
# MyModel.model.solve(GUROBI())

Stability constraints...
Capacity constraints...

Average rank before optimization: 1.3564500000000002.


Constraints with auxiliary z...
FOSD constraints...
Set parameter Username
Set parameter LicenseID to value 2595965
Academic license - for non-commercial use only - expires 2025-12-05
Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (win64 - Windows 11.0 (22631.2))

CPU model: 11th Gen Intel(R) Core(TM) i7-1185G7 @ 3.00GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 43126 rows, 20401 columns and 257041 nonzeros
Model fingerprint: 0x1fbfbc3f
Variable types: 10301 continuous, 10100 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 8e+00]
  Objective range  [5e-02, 3e-01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [5e-01, 8e+00]
Presolve removed 29743 rows and 14207 columns
Presolve time: 0.70s
Presolved: 13383 rows, 6194 columns, 49766 nonzeros
Variable types: 

KeyboardInterrupt: 

Exception ignored in: 'gurobipy._core.logcallbackstub'
Traceback (most recent call last):
  File "C:\Users\tdemeule\AppData\Local\anaconda3\Lib\site-packages\ipykernel\iostream.py", line 624, in write
    def write(self, string: str) -> Optional[int]:  # type:ignore[override]

KeyboardInterrupt: 


 265425 178097    1.34125  268    2    1.35165    1.25000  7.52%  18.0  535s


KeyboardInterrupt: 

Exception ignored in: 'gurobipy._core.logcallbackstub'
Traceback (most recent call last):
  File "C:\Users\tdemeule\AppData\Local\anaconda3\Lib\site-packages\ipykernel\iostream.py", line 624, in write
    def write(self, string: str) -> Optional[int]:  # type:ignore[override]

KeyboardInterrupt: 


 271227 182979    1.27500  122   15    1.35165    1.25000  7.52%  18.0  545s
 274408 185518    1.30000  196    5    1.35165    1.25000  7.52%  17.9  550s
 276778 187501    1.30000  158    3    1.35165    1.25000  7.52%  17.9  555s
 280843 190793    1.30000  167    5    1.35165    1.25000  7.52%  17.8  560s


In [ ]:
MyModel.print_solution()

In [ ]:
# Visualize the solution
q.visualize()


In [ ]:
# Asses the difference
diff = Assignment(MyData, q.assignment - p, "Ex_paper_Diff")
diff.visualize()

In [422]:
print(gale_shapley(MyData))

[[0. 0. 1. 0.]
 [0. 0. 0. 1.]
 [0. 1. 0. 0.]
 [1. 0. 0. 0.]]


In [735]:
int(MyData.pref[6][0]) -1
MyData.ID_school.index(MyData.pref[0][0])
MyData.ID_school
MyData.ID_school.index(MyData.pref[0][0])

8

## Appendix: minimal working code pulp

In [ ]:
# Example of a simple MILP formulation in Gurobi 

from pulp import LpProblem, LpVariable, LpMaximize, GUROBI

# Define a simple problem
prob = LpProblem("SimpleProblem", LpMaximize)

# Define variables
x = LpVariable("x", lowBound=0)  # x >= 0
y = LpVariable("y", lowBound=0)  # y >= 0

# Objective Function
prob += 3 * x + 2 * y, "Objective"

# Constraints
prob += 2 * x + y <= 20, "Constraint 1"
prob += 4 * x + 3 * y <= 50, "Constraint 2"

# Solve using Gurobi API
prob.solve(GUROBI())

# Print the results
print(f"Status: {prob.status}")
print(f"x = {x.varValue}")
print(f"y = {y.varValue}")
prob.writeLP("Test.lp")
